In [1]:
import pandas as pd
import re
import random

random.seed(42)
from tqdm.auto import tqdm
tqdm.pandas()

/share/u/arunas/miniconda3/envs/broca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Genders for subjects based on italian

In [2]:
genderDict = {
    "pear": "kar",
    "author": "kon",
    "authors": "kons",
    "banana": "kar",
    "biscuit": "kon",
    "book": "kon",
    "bottle": "kar",
    "box": "kar",
    "boy": "kon",
    "boys": "kons",
    "bulb": "kar",
    "cabinet": "kar",
    "cap": "kon",
    "cat": "kon",
    "cats": "kons",
    "chapter": "kon",
    "chalk": "kon",
    "cup": "kar",
    "cucumber": "kon",
    "dog": "kon",
    "dogs": "kons",
    "fish": "kon",
    "fruit": "kar",
    "girl": "kar",
    "girls": "kars",
    "hill": "kar",
    "man": "kon",
    "men": "kons",
    "meal": "kon",
    "mountain": "kar",
    "mouse": "kon",
    "newspaper": "kon",
    "pizza": "kar",
    "poet": "kon",
    "poets": "kons",
    "poem": "kar",
    "rock": "kon",
    "roof": "kon",
    "speaker": "kon",
    "speakers": "kons",
    "staircase": "kar",
    "story": "kar",
    "teacher": "kon",
    "teachers": "kons",
    "toy": "kon",
    "tree": "kar",
    "woman": "kar",
    "women": "kars",
    "writer": "kon",
    "writers": "kons"
}

pastTense = {
    'climbs' : 'climbed',
    'reads': 'read',
    'carries': 'carried',
    'eats': 'ate',
    'holds': 'held',
    'takes' :'took',
    'brings': 'brought',
    'reads': 'read',
    'climb' : 'climbed',
    'read': 'read',
    'carry': 'carried',
    'eat': 'ate',
    'hold': 'held',
    'take' :'took',
    'bring': 'brought',
    'read': 'read'
}

infinitive = {
    'climbs' : 'to climb',
    'reads': 'to read',
    'carries': 'to carry',
    'eats': 'to eat',
    'holds': 'to hold',
    'takes' : 'to take',
    'brings': 'to bring',
    'reads': 'to read',
    'climb' : 'to climb',
    'read': 'to read',
    'carry': 'to carry',
    'eat': 'to eat',
    'hold': 'to hold',
    'take' : 'to take',
    'bring': 'to bring',
    'read': 'to read'
}

pluralObjects = {
    'fish': 'fish',
    'mouse': 'mice',
    'bottle': 'bottles',
    'newspaper': 'newspapers',
    'chalk': 'chalks',
    'box': 'boxes',
    'cap': 'caps',
    'bulb': 'bulbs',
    'cup': 'cups',
    'toy': 'toys',
    'staircase': 'staircases',
    'rock': 'rocks',
    'hill': 'hills',
    'mountain': 'mountains',
    'roof': 'roofs',
    'tree': 'trees',
    'biscuit': 'biscuits',
    'banana': 'bananas',
    'pear': 'pears',
    'meal': 'meals',
    'fruit': 'fruits',
    'cucumber': 'cucumbers',
    'pizza': 'pizzas',
    'book': 'books',
    'poem': 'poems',
    'story': 'stories',
    'chapter': 'chapters'
}

passiveSeed = {
    'carries': 'carried',
    'carry': 'carried',
    'holds': 'held',
    'hold': 'held',
    'takes': 'taken',
    'take': 'taken',
    'brings': 'brought',
    'bring': 'brought',
    'climbs': 'climbed',
    'climb': 'climbed',
    'eats': 'eaten',
    'eat': 'eaten',
    'reads': 'read',
    'read': 'read'
}


In [3]:
it_genderDict = {
    "pera": ["la", "una"],
    "scrittore": ["lo", "uno"],
    "scrittori": ["gli"],
    "banana": ["la", "una"],
    "biscotto": ["il", "un"],
    "libro": ["il", "un"],
    "bottiglia": ["la", "una"],
    "scatola": ["la", "una"],
    "ragazzo": ["il", "un"],
    "ragazzi": ["i"],
    "lampadina": ["la", "una"],
    "credenza": ["la", "una"],
    "cappello": ["il", "un"],
    "gatto": ["il", "un"],
    "gatti": ["i"],
    "capitolo": ["il", "un"],
    "gesso": ["il", "un"],
    "tazza": ["la", "una"],
    "cetriolo": ["il", "un"],
    "cane": ["il", "un"],
    "cani": ["i"],
    "oratorio": ["il", "un"],
    "pesce": ["il", "un"],
    "frutta": ["la", "una"],
    "ragazza": ["la", "una"],
    "ragazze": ["le"],
    "collina": ["la", "una"],
    "uomo": ["l'", "un'"],
    "uomini": ["gli"],
    "pasto": ["il", "un"],
    "montagna": ["la", "una"],
    "topo": ["il", "un"],
    "giornale": ["il", "un"],
    "pizza": ["la", "una"],
    "poeta": ["il", "un"],
    "poeti": ["i"],
    "poema": ["il", "un"],
    "roccia": ["la", "una"],
    "tetto": ["il", "un"],
    "oratore": ["l'", "un'"],
    "oratori": ["gli"],
    "scala": ["la", "una"],
    "storia": ["la", "una"],
    "insegnante": ["l'", "un'"],
    "insegnanti": ["gli"],
    "giocattolo": ["il", "un"],
    "albero": ["l'", "un'"],
    "donna": ["la", "una"],
    "donne": ["le"],
    "autore": ["l'", "un'"],
    "autori": ["gli"]
}

it_pastTense = {
    'scala' : { 'la': 'ha scalata', 'il': 'ha scalato'},
    'legge': { 'la': 'ha letto', 'il': 'ha letto' },
    'porta': { 'la': 'ha portato', 'il': 'ha portato' },
    'mangia': { 'la': 'ha mangiato', 'il': 'ha mangiato' },
    'tiene': { 'la': 'ha tenuto', 'il': 'ha tenuto' },
    'prende' : { 'la': 'ha preso', 'il': 'ha preso'}
}

it_infinitive = {
    'scala' : 'scalare',
    'legge': 'leggere',
    'porta': 'portare',
    'mangia': 'mangiare',
    'tiene': 'tenere',
    'prende' : 'prendere'
}

it_pluralObjects = {
    'pesce': 'pesci',
    'topo': 'topi',
    'bottiglia': 'bottiglie',
    'giornale': 'giornali',
    'gesso': 'gessi',
    'scatola': 'scatole',
    'cappello': 'cappelli',
    'lampadina': 'lampadine',
    'tazza': 'tazze',
    'giocattolo': 'giocattoli',
    'scala': 'scale',
    'roccia': 'rocce',
    'collina': 'colline',
    'montagna': 'montagne',
    'tetto': 'tetti',
    'albero': 'alberi',
    'biscotto': 'biscotti',
    'banana': 'banane',
    'pera': 'pere',
    'pasto': 'pasti',
    'frutta': 'frutta',
    'cetriolo': 'cetrioli',
    'pizza': 'pizze',
    'libro': 'libri',
    'poema': 'poemi',
    'storia': 'storie',
    'capitolo': 'capitolo'
}

it_passiveSeed = {
    'scala' : { 'la': 'è scalata', 'una': 'è scalata', 'il': 'è scalato', 'un': 'è scalato', 'uno': 'è scalato'},
    'legge': { 'la': 'è letto', 'una': 'è letto', 'il': 'è letto' , 'un': 'è letto', 'uno': 'è letto' },
    'porta': { 'la': 'è portato', 'una': 'è portato', 'il': 'è portato' , 'un': 'è portato', 'uno': 'è portato' },
    'mangia': { 'la': 'è mangiato', 'una': 'è mangiato', 'il': 'è mangiato' , 'un': 'è mangiato', 'uno': 'è mangiato' },
    'tiene': { 'la': 'è tenuto', 'una': 'è tenuto', 'il': 'è tenuto' , 'un': 'è tenuto', 'uno': 'è tenuto' },
    'prende' : { 'la': 'è preso', 'una': 'è preso', 'il': 'è preso', 'un': 'è preso', 'uno': 'è preso'},
    'salgono' : { 'la': 'è scalata', 'una': 'è scalata', 'il': 'è scalato', 'un': 'è scalato', 'uno': 'è scalato'},
    'leggono': { 'la': 'è letto', 'una': 'è letto', 'il': 'è letto' , 'un': 'è letto', 'uno': 'è letto' },
    'portano': { 'la': 'è portato', 'una': 'è portato', 'il': 'è portato' , 'un': 'è portato', 'uno': 'è portato' },
    'mangiano': { 'la': 'è mangiato', 'una': 'è mangiato', 'il': 'è mangiato' , 'un': 'è mangiato', 'uno': 'è mangiato' },
    'tengono': { 'la': 'è tenuto', 'una': 'è tenuto', 'il': 'è tenuto' , 'un': 'è tenuto', 'uno': 'è tenuto' },
    'prendono' : { 'la': 'è preso', 'una': 'è preso', 'il': 'è preso', 'un': 'è preso', 'uno': 'è preso'}
}


nounDict_jp = {
    "pear": "梨",
    "author": "著者",
    "banana": "バナナ",
    "biscuit": "ビスケット",
    "book": "本",
    "bottle": "ボトル",
    "box": "箱",
    "boy": "男の子",
    "bulb": "電球",
    "cap": "帽子",
    "cat": "猫",
    "chapter": "章",
    "chalk": "白亜",
    "cup": "コップ",
    "cucumber": "胡瓜",
    "dog": "犬",
    "fish": "魚",
    "fruit": "果物",
    "girl": "女の子",
    "hill": "丘",
    "man": "男",
    "meal": "食事",
    "mountain": "山",
    "mouse": "マウス",
    "newspaper": "新聞",
    "noodles": "麺",
    "poet": "詩人",
    "poem": "詩",
    "rock": "岩石",
    "roof": "屋根",
    "speaker": "スピーカー",
    "staircase": "階段",
    "story": "小説",
    "teacher": "先生",
    "toy": "玩具",
    "tree": "木",
    "woman": "女",
    "writer": "著者",
    "pizza": "ピザ",
    "pears": "梨",
    "authors": "著者",
    "bananas": "バナナ",
    "biscuits": "ビスケット",
    "books": "本",
    "bottles": "ボトル",
    "boxes": "箱",
    "boys": "男の子",
    "bulbs": "電球",
    "caps": "帽子",
    "cats": "猫",
    "chapters": "章",
    "chalks": "白亜",
    "cups": "コップ",
    "cucumbers": "胡瓜",
    "dogs": "犬",
    "fish": "魚",
    "fruits": "果物",
    "girls": "女の子",
    "hills": "丘",
    "men": "男性",
    "meals": "食事",
    "mountains": "山",
    "mouses": "マウス",
    "newspapers": "新聞",
    "noodles": "麺",
    "poets": "詩人",
    "poems": "詩",
    "rocks": "岩石",
    "roofs": "屋根",
    "speakers": "スピーカー",
    "staircases": "階段",
    "stories": "小説",
    "teachers": "先生",
    "toys": "玩具",
    "trees": "木",
    "women": "女性",
    "writers": "著者",
    "pizzas": "ピザ"
}

verbDict_jp = {
    "eats": "食べる",
    "reads": "読む",
    "carries": "運ぶ",
    "climbs": "のぼる",
    "takes": "とる",
    "holds": "持つ",
    "brings": "もたらす",
     "eat": "食べる",
    "read": "読む",
    "carry": "運ぶ",
    "climb": "のぼる",
    "take": "とる",
    "hold": "持つ",
    "bring": "もたらす"
}

passiveDict_jp = {
    "食べる": "食べられる",
    "読む": "読まれる",
    "運ぶ": "運ばれる",
    "のぼる": "のぼられる",
    "とる": "とられる",
    "持つ": "持たれる",
    "もたらす": "持たれる"
    
}

particleDict_jp = {
    "wa": "は",
    "ga": "が",
    "o": "を",
    "to": "と",
    "ni": "に",
    "nai": "ない",
    "ta": "た"
}

# Generate sentences

In [4]:
seed = [ { 'verb' : ['carries', 'holds', 'takes', 'brings'],  'subject': ['dog', 'cat', 'man', 'woman', 'teacher', 'girl', 'boy'], 'object': ['fish', 'mouse', 'bottle', 'newspaper', 'chalk', 'box', 'cap', 'bulb', 'cup', 'toy']},

{ 'verb': ['climbs'], 'subject': ['dog', 'cat', 'man', 'woman', 'teacher', 'girl', 'boy'], 'object': ['staircase', 'rock', 'hill', 'mountain', 'roof', 'tree'] },

{ 'verb': ['eats'], 'subject' : ['dog', 'cat', 'man', 'woman', 'teacher', 'girl', 'boy'], 'object': ['biscuit', 'fish', 'banana', 'pear', 'meal', 'fruit', 'cucumber', 'pizza' ]},
{'verb': ['reads'], 'subject' : ['poet', 'author', 'writer', 'speaker', 'teacher', 'girl', 'boy'], 'object': ['book', 'poem', 'story', 'chapter']} ]

subordinateSeed = [ { 'verb' : ['sees', 'says', 'notices', 'states', 'claims'],  'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]


it_seed = [ { 'verb' : ['porta', 'tiene', 'prende', 'porta'],  'subject': ['cane', 'gatto', 'uomo', 'donna', 'insegnante', 'ragazza', 'ragazzo'], 'object': ['pesce', 'topo', 'bottiglia', 'giornale', 'gesso', 'scatola', 'cappello', 'lampadina', 'tazza', 'giocattolo']},

{ 'verb': ['scala'], 'subject': ['cane', 'gatto', 'uomo', 'donna', 'insegnante', 'ragazza', 'ragazzo'], 'object': ['scala', 'roccia', 'collina', 'montagna', 'tetto', 'albero'] },

{ 'verb': ['mangia'], 'subject' : ['cane', 'gatto', 'uomo', 'donna', 'insegnante', 'ragazza', 'ragazzo'], 'object': ['biscotto', 'pesce', 'banana', 'pera', 'pasto', 'frutta', 'cetriolo', 'pizza' ]},
{'verb': ['legge'], 'subject' : ['poeta', 'autore', 'scrittore', 'oratorio', 'insegnante', 'ragazza', 'ragazzo'], 'object': ['libro', 'poema', 'storia', 'capitolo']} ]

it_subordinateSeed = [ { 'verb' : ['vede', 'dice', 'osserva', 'sa', 'afferma'],  'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]

df = pd.DataFrame()

for oidx, obj in enumerate(seed):
    for sidx, subj in enumerate(obj['subject']):
        for obidx, ob in enumerate(obj['object']):
            for vidx, verb in enumerate(obj['verb']):
                sdet = random.choice(['the', 'a'])
                odet = random.choice(['the', 'a'])
                pSubj = random.choice(subordinateSeed[0]['subject'])
                pVerb = random.choice(subordinateSeed[0]['verb'])

                if sdet == 'the':
                    it_sdet = it_genderDict[it_seed[oidx]['subject'][sidx]][0] 
                else: 
                    it_sdet = it_genderDict[it_seed[oidx]['subject'][sidx]][1]

                if odet == 'the':
                    it_odet = it_genderDict[it_seed[oidx]['object'][obidx]][0] 
                else: 
                    it_odet = it_genderDict[it_seed[oidx]['object'][obidx]][1]
                
                it_subj = it_seed[oidx]['subject'][sidx]
                it_ob = it_seed[oidx]['object'][obidx]
                it_verb = it_seed[oidx]['verb'][vidx]
                
                it_pSubj = random.choice(it_subordinateSeed[0]['subject'])
                it_pVerb = random.choice(it_subordinateSeed[0]['verb'])

                temp_odet = it_odet
                if it_odet =='il':
                    ita_passive_from = 'dal'
                if it_odet =='la':
                    ita_passive_from = 'dalla'
                elif it_odet == "un" or it_odet == "un'":
                    ita_passive_from = 'da un'
                    temp_odet = 'un'
                elif it_odet == "una":
                    ita_passive_from = 'da una'
                elif it_odet == "l'":
                    temp_odet = 'il'
                    ita_passive_from = "dall'"
                ita_passive_sentence = f"{it_odet} {it_ob} {it_passiveSeed[it_verb][temp_odet]} {ita_passive_from} {it_subj}"

                if (it_sdet != "l'" and it_odet != "l'"):
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}",
                        "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-3-passive":ita_passive_sentence,
                        "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                        "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]),
                        "ita-u-3-gender":f"{it_odet} {it_subj} {it_verb} {it_odet} {it_ob}",
                        "en": f"{sdet} {subj} {verb} {odet} {ob}",
                        "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                        "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}",
                        "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                        "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                        "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?"
                    }])])
                elif (it_sdet == "l'" and it_odet != "l'"):
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}",
                        "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-3-passive":ita_passive_sentence,
                        "ita-u-1-negation": f"{it_sdet}{it_subj} {it_verb} {it_odet} no {it_ob}",
                        "ita-u-2-invert": " ".join(f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]),
                        "ita-u-3-gender": f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}", #using sdet instead of odet for subj, because it's a word starting w a vowel, and therefore the det is not gendered
                        "en": f"{sdet} {subj} {verb} {odet} {ob}",
                        "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                        "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}",
                        "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                        "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                        "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?"
                    }])])
                elif (it_sdet == "l'" and it_odet == "l'"):
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}",
                        "ita-r-1-null_subject": f"{it_verb} {it_odet}{it_ob}",
                        "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}",
                        "ita-r-3-passive":ita_passive_sentence, 
                        "ita-u-1-negation": f"{it_sdet}{it_subj} {it_verb} il no {it_ob}",
                        "ita-u-2-invert": " ".join( f"{it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}".split(" ")[::-1]),
                        "ita-u-3-gender": f"{it_odet}{it_subj} {it_verb} {it_odet}{it_ob}", 
                        "en": f"{sdet} {subj} {verb} {odet} {ob}", 
                        "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}", 
                        "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}",
                        "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                        "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                        "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?"
                    }])])
                else:
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}",
                        "ita-r-1-null_subject": f"{it_verb} {it_odet}{it_ob}",
                        "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}",
                        "ita-r-3-passive":ita_passive_sentence, 
                        "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                        "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}".split(" ")[::-1]),
                        "ita-u-3-gender": f"il {it_subj} {it_verb} {it_odet}{it_ob}",
                        "en": f"{sdet} {subj} {verb} {odet} {ob}",
                        "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                        "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}", 
                        "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                        "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                        "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?"
                    }])])
                

In [5]:

pluralSeed = [ { 'verb' : ['carry', 'hold', 'take', 'bring'],  'subject': ['dogs', 'cats', 'men', 'women', 'teachers', 'girls', 'boys'], 'object': ['fish', 'mouse', 'bottle', 'newspaper', 'chalk', 'box', 'cap', 'bulb', 'cup', 'toy']},

{ 'verb': ['climb'], 'subject': ['dogs', 'cats', 'men', 'women', 'teachers', 'girls', 'boys'], 'object': ['staircase', 'rock', 'hill', 'mountain', 'roof', 'tree'] },

{ 'verb': ['eat'], 'subject' : ['dogs', 'cats', 'men', 'women', 'teachers', 'girls', 'boys'], 'object': ['biscuit', 'fish', 'banana', 'pear', 'meal', 'fruit', 'cucumber', 'pizza' ] },
{'verb': ['read'], 'subject' : ['poets', 'authors', 'writers', 'speakers', 'teachers', 'girls', 'boys'], 'object': ['book', 'poem', 'story', 'chapter']} ]

it_pluralSeed = [ { 'verb' : ['portano', 'tengono', 'prendono', 'portano'],  'subject': ['cani', 'gatti', 'uomini', 'donne', 'insegnanti', 'ragazze', 'ragazzi'], 'object': ['pesce', 'topo', 'bottiglia', 'giornale', 'gesso', 'scatola', 'cappello', 'lampadina', 'tazza', 'giocattolo']},

{ 'verb': ['salgono'], 'subject': ['cani', 'gatti', 'uomini', 'donne', 'insegnanti', 'ragazze', 'ragazzi'], 'object': ['scala', 'roccia', 'collina', 'montagna', 'tetto', 'albero'] },

{ 'verb': ['mangiano'], 'subject' : ['cani', 'gatti', 'uomini', 'donne', 'insegnanti', 'ragazze', 'ragazzi'], 'object': ['biscotto', 'pesce', 'banana', 'pera', 'pasto', 'frutta', 'cetriolo', 'pizza' ] },
{'verb': ['leggono'], 'subject' : ['poeti', 'autori', 'scrittori', 'oratori', 'insegnanti', 'ragazze', 'ragazzi'], 'object': ['libro', 'poema', 'storia', 'capitolo']} ]

it_subordinateSeed = [ { 'verb' : ['vede', 'dice', 'osserva', 'sa', 'afferma'],  'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]

for oidx, obj in enumerate(pluralSeed):
    for sidx, subj in enumerate(obj['subject']):
        for obidx, ob in enumerate(obj['object']):
            for vidx, verb in enumerate(obj['verb']):
                pSubj = random.choice(subordinateSeed[0]['subject'])
                pVerb = random.choice(subordinateSeed[0]['verb'])
                odet = random.choice(['the', 'a'])
                
                it_subj = it_pluralSeed[oidx]['subject'][sidx]
                it_sdet = it_genderDict[it_subj][0]
                it_ob = it_pluralSeed[oidx]['object'][obidx]
                it_odet = it_genderDict[it_ob][0]

                it_verb = it_pluralSeed[oidx]['verb'][vidx]
                
                it_pSubj = random.choice(it_subordinateSeed[0]['subject'])
                it_pVerb = random.choice(it_subordinateSeed[0]['verb'])

                temp_sdet = it_sdet
                if it_sdet == 'i':
                    temp_sdet = 'il'
                    ita_passive_from = 'dai'
                if it_sdet == 'le':
                    temp_sdet = 'la'
                    ita_passive_from = 'dalle'
                elif it_sdet == "gli":
                    temp_sdet = 'il'
                    ita_passive_from = "dagli'"
                ita_passive_sentence = f"{it_odet} {it_ob} {it_passiveSeed[it_verb][temp_sdet]} {ita_passive_from} {it_subj}"

                if it_odet == 'l':
                    temp_odet = 'il'
                else:
                    temp_odet = it_odet
                    
                df = pd.concat([df, pd.DataFrame.from_dict([{
                    "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}", 
                    "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}", 
                    "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                    "ita-r-3-passive": ita_passive_sentence,
                    "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                    "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]), 
                    "ita-u-3-gender": f"{temp_odet} {it_subj} {it_verb} {it_odet} {it_ob}",
                    "en": f"the {subj} {verb} {odet} {ob}",
                    "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                    "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by the {subj}",
                    "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                    "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                    "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?"
                }])])
                if (f"{odet} {ob} is {passiveSeed[verb]} by the {subj}" == None):
                    print(f"{odet} {ob} is {passiveSeed[verb]} by the {subj}", odet, ob, passiveSeed[verb], subj)

df.reset_index()

,index,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,en,en-r-1-subordinate,en-r-2-passive,en-u-1-negation,en-u-2-inversion,en-u-3-qsubordinate
0,0,il cane porta il pesce,porta il pesce,Leela dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog carries the fish,Tom notices that the dog carries the fish,the fish is carried by the dog,the dog carries the doesn't fish,fish the carries dog the,Tom notices that does the dog carries the fish?
1,0,il cane tiene il pesce,tiene il pesce,Sheela afferma che il cane tiene il pesce,il pesce è tenuto dal cane,il cane tiene il no pesce,pesce il tiene cane il,il cane tiene il pesce,the dog holds the fish,Tom claims that the dog holds the fish,the fish is held by the dog,the dog holds the doesn't fish,fish the holds dog the,Tom claims that does the dog holds the fish?
2,0,un cane prende il pesce,prende il pesce,Leela dice che un cane prende il pesce,il pesce è preso dal cane,un cane prende il no pesce,pesce il prende cane un,il cane prende il pesce,a dog takes the fish,Sheela sees that the dog takes the fish,the fish is taken by a dog,a dog takes the doesn't fish,fish the takes dog a,Sheela sees that does the dog takes the fish?
3,0,il cane porta il pesce,porta il pesce,Gomu dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog brings the fish,Tom claims that the dog brings the fish,the fish is brought by the dog,the dog brings the doesn't fish,fish the brings dog the,Tom claims that does the dog brings the fish?
4,0,un cane porta un topo,porta un topo,Harry dice che un cane porta un topo,un topo è portato da un cane,un cane porta un no topo,topo un porta cane un,un cane porta un topo,a dog carries a mouse,Harry sees that the dog carries the mouse,a mouse is carried by a dog,a dog carries a doesn't mouse,mouse a carries dog a,Harry sees that does the dog carries the mouse?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,0,le ragazze leggono il capitolo,leggono il capitolo,Maria sa che le ragazze leggono il capitolo,il capitolo è letto dalle ragazze,le ragazze leggono il no capitolo,capitolo il leggono ragazze le,il ragazze leggono il capitolo,the girls read a chapter,Tom says that the girls read the chapter,a chapter is read by the girls,a girls read a doesn't chapter,chapter a read girls a,Tom says that does the girls read the chapter?
808,0,i ragazzi leggono il libro,leggono il libro,Maria osserva che i ragazzi leggono il libro,il libro è letto dai ragazzi,i ragazzi leggono il no libro,libro il leggono ragazzi i,il ragazzi leggono il libro,the boys read the book,John notices that the boys read the book,the book is read by the boys,a boys read the doesn't book,book the read boys a,John notices that does the boys read the book?
809,0,i ragazzi leggono il poema,leggono il poema,Gomu sa che i ragazzi leggono il poema,il poema è letto dai ragazzi,i ragazzi leggono il no poema,poema il leggono ragazzi i,il ragazzi leggono il poema,the boys read a poem,Leela sees that the boys read the poem,a poem is read by the boys,a boys read a doesn't poem,poem a read boys a,Leela sees that does the boys read the poem?
810,0,i ragazzi leggono la storia,leggono la storia,Harry sa che i ragazzi leggono la storia,la storia è letto dai ragazzi,i ragazzi leggono la no storia,storia la leggono ragazzi i,la ragazzi leggono la storia,the boys read a story,Maria states that the boys read the story,a story is read by the boys,a boys read a doesn't story,story a read boys a,Maria states that does the boys read the story?


## Italian sentence

In [6]:
df['it'] = df.progress_apply(lambda row: " ".join([genderDict[row['en'].split(" ")[1]]] + row['en'].split(" ")[1:3] + [genderDict[row['en'].split(" ")[4]]] + [row['en'].split(" ")[-1]]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 112691.91it/s]


## IT Real grammar 1 (Null Subject parameter)

In [7]:
df['it-r-1-null_subject'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[2:]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 155942.07it/s]


## IT Real Grammar 2 (Passive construction)

In [8]:
df['it-r-2-passive'] =  df.progress_apply(lambda row: " ".join([genderDict[row['en-r-2-passive'].split(" ")[1]]] + row['en-r-2-passive'].split(" ")[1:-2] + [genderDict[row['en-r-2-passive'].split(" ")[-1]]] + [row['en-r-2-passive'].split(" ")[-1]]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 125267.58it/s]


## IT Real Grammar 3 (Subordinate construction)

In [9]:
df['it-r-3-subordinate'] =  df.progress_apply(lambda row: " ".join(row['en-r-1-subordinate'].split(" ")[0:3] + row['it'].split(" ")), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 189346.46it/s]


In [10]:
df

,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,en,en-r-1-subordinate,en-r-2-passive,en-u-1-negation,en-u-2-inversion,en-u-3-qsubordinate,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate
0,il cane porta il pesce,porta il pesce,Leela dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog carries the fish,Tom notices that the dog carries the fish,the fish is carried by the dog,the dog carries the doesn't fish,fish the carries dog the,Tom notices that does the dog carries the fish?,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Tom notices that kon dog carries kon fish
0,il cane tiene il pesce,tiene il pesce,Sheela afferma che il cane tiene il pesce,il pesce è tenuto dal cane,il cane tiene il no pesce,pesce il tiene cane il,il cane tiene il pesce,the dog holds the fish,Tom claims that the dog holds the fish,the fish is held by the dog,the dog holds the doesn't fish,fish the holds dog the,Tom claims that does the dog holds the fish?,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,Tom claims that kon dog holds kon fish
0,un cane prende il pesce,prende il pesce,Leela dice che un cane prende il pesce,il pesce è preso dal cane,un cane prende il no pesce,pesce il prende cane un,il cane prende il pesce,a dog takes the fish,Sheela sees that the dog takes the fish,the fish is taken by a dog,a dog takes the doesn't fish,fish the takes dog a,Sheela sees that does the dog takes the fish?,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Sheela sees that kon dog takes kon fish
0,il cane porta il pesce,porta il pesce,Gomu dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog brings the fish,Tom claims that the dog brings the fish,the fish is brought by the dog,the dog brings the doesn't fish,fish the brings dog the,Tom claims that does the dog brings the fish?,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,Tom claims that kon dog brings kon fish
0,un cane porta un topo,porta un topo,Harry dice che un cane porta un topo,un topo è portato da un cane,un cane porta un no topo,topo un porta cane un,un cane porta un topo,a dog carries a mouse,Harry sees that the dog carries the mouse,a mouse is carried by a dog,a dog carries a doesn't mouse,mouse a carries dog a,Harry sees that does the dog carries the mouse?,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Harry sees that kon dog carries kon mouse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,le ragazze leggono il capitolo,leggono il capitolo,Maria sa che le ragazze leggono il capitolo,il capitolo è letto dalle ragazze,le ragazze leggono il no capitolo,capitolo il leggono ragazze le,il ragazze leggono il capitolo,the girls read a chapter,Tom says that the girls read the chapter,a chapter is read by the girls,a girls read a doesn't chapter,chapter a read girls a,Tom says that does the girls read the chapter?,kars girls read kon chapter,read kon chapter,kon chapter is read by kars girls,Tom says that kars girls read kon chapter
0,i ragazzi leggono il libro,leggono il libro,Maria osserva che i ragazzi leggono il libro,il libro è letto dai ragazzi,i ragazzi leggono il no libro,libro il leggono ragazzi i,il ragazzi leggono il libro,the boys read the book,John notices that the boys read the book,the book is read by the boys,a boys read the doesn't book,book the read boys a,John notices that does the boys read the book?,kons boys read kon book,read kon book,kon book is read by kons boys,John notices that kons boys read kon book
0,i ragazzi leggono il poema,leggono il poema,Gomu sa che i ragazzi leggono il poema,il poema è letto dai ragazzi,i ragazzi leggono il no poema,poema il leggono ragazzi i,il ragazzi leggono il poema,the boys 

## IT Unreal Grammar 1: Add a negation after the 3rd word in the nullified subject sentence

In [11]:
df['it-u-1-negation'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[:4] + [ "no" ] + row['en'].split(" ")[4:]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 181360.82it/s]


## IT Unreal Grammar 2: Invert italian sentence

In [12]:
df['it-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[::-1]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 161052.39it/s]


## IT Unreal Grammar 3: Same gender for subject and object

In [13]:
df['it-u-3-gender'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[:3] + [row['it'].split(" ")[0]] + [row['it'].split(" ")[-1]]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 156400.39it/s]


## JP real grammar 1 (Wa after subj, o after obj, verb)

In [14]:
df['jp-r-1-sov'] = df.progress_apply(lambda row: " ".join(row["en"].split(" ")[:2]) + " wa " + " ".join(row["en"].split(" ")[-2:]) + " o " + row["en"].split(" ")[2], axis=1)

df['jap-r-1-sov'] = df.progress_apply(lambda row: f'{nounDict_jp[row["jp-r-1-sov"].split(" ")[1]]} {particleDict_jp["wa"]} {nounDict_jp[row["jp-r-1-sov"].split(" ")[4]]} {particleDict_jp["o"]} {verbDict_jp[row["jp-r-1-sov"].split(" ")[-1]]}' , axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 150878.25it/s]


## JP real grammar 2 (Passive construction)

In [15]:
df['jp-r-2-passive'] = df.progress_apply(lambda row: " ".join(row["en"].split(" ")[3:5]) + " wa " + " ".join(row["en"].split(" ")[:2]) + " ni " + infinitive[row["en"].split(" ")[2]] + " reru", axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 151018.75it/s]


In [16]:
df

,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,en,en-r-1-subordinate,en-r-2-passive,...,en-u-3-qsubordinate,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate,it-u-1-negation,it-u-2-invert,it-u-3-gender,jp-r-1-sov,jp-r-2-passive
0,il cane porta il pesce,porta il pesce,Leela dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog carries the fish,Tom notices that the dog carries the fish,the fish is carried by the dog,...,Tom notices that does the dog carries the fish?,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Tom notices that kon dog carries kon fish,kon dog carries kon no fish,fish kon carries dog kon,kon dog carries kon fish,the dog wa the fish o carries,the fish wa the dog ni to carry reru
0,il cane tiene il pesce,tiene il pesce,Sheela afferma che il cane tiene il pesce,il pesce è tenuto dal cane,il cane tiene il no pesce,pesce il tiene cane il,il cane tiene il pesce,the dog holds the fish,Tom claims that the dog holds the fish,the fish is held by the dog,...,Tom claims that does the dog holds the fish?,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,Tom claims that kon dog holds kon fish,kon dog holds kon no fish,fish kon holds dog kon,kon dog holds kon fish,the dog wa the fish o holds,the fish wa the dog ni to hold reru
0,un cane prende il pesce,prende il pesce,Leela dice che un cane prende il pesce,il pesce è preso dal cane,un cane prende il no pesce,pesce il prende cane un,il cane prende il pesce,a dog takes the fish,Sheela sees that the dog takes the fish,the fish is taken by a dog,...,Sheela sees that does the dog takes the fish?,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Sheela sees that kon dog takes kon fish,kon dog takes kon no fish,fish kon takes dog kon,kon dog takes kon fish,a dog wa the fish o takes,the fish wa a dog ni to take reru
0,il cane porta il pesce,porta il pesce,Gomu dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog brings the fish,Tom claims that the dog brings the fish,the fish is brought by the dog,...,Tom claims that does the dog brings the fish?,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,Tom claims that kon dog brings kon fish,kon dog brings kon no fish,fish kon brings dog kon,kon dog brings kon fish,the dog wa the fish o brings,the fish wa the dog ni to bring reru
0,un cane porta un topo,porta un topo,Harry dice che un cane porta un topo,un topo è portato da un cane,un cane porta un no topo,topo un porta cane un,un cane porta un topo,a dog carries a mouse,Harry sees that the dog carries the mouse,a mouse is carried by a dog,...,Harry sees that does the dog carries the mouse?,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Harry sees that kon dog carries kon mouse,kon dog carries kon no mouse,mouse kon carries dog kon,kon dog carries kon mouse,a dog wa a mouse o carries,a mouse wa a dog ni to carry reru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,le ragazze leggono il capitolo,leggono il capitolo,Maria sa che le ragazze leggono il capitolo,il capitolo è letto dalle ragazze,le ragazze leggono il no capitolo,capitolo il leggono ragazze le,il ragazze leggono il capitolo,the girls read a chapter,Tom says that the girls read the chapter,a chapter is read by the girls,...,Tom says that does the girls read the chapter?,kars girls read kon chapter,read kon chapter,kon chapter is read by kars girls,Tom says that kars girls read kon chapter,kars girls read kon no chapter,chapter kon read girls kars,kars girls read kars chapter,the girls wa a chapter o read,a chapter wa the girls ni to read reru
0,i ragazzi leggono il libro,leggono il libro,Maria osserva che i ragazzi leggono il libro,il libro è l

## JP real grammar 3 (Subordinate construction)

In [17]:
df['jp-r-3-subordinate'] = df.progress_apply(lambda row: " ".join([row["en-r-1-subordinate"].split(" ")[0]] + ["wa"] + row["en-r-1-subordinate"].split(" ")[3:5] + ["ga"] + row["en-r-1-subordinate"].split(" ")[-2:]  + ["o"] + [row["en-r-1-subordinate"].split(" ")[5]] + ["to"] + [row["en-r-1-subordinate"].split(" ")[1]]), axis=1)

# df['jap-r-3-subordinate'] = df.progress_apply(lambda row: f'{nounDict_jp[row["jp-r-2-passive"].split(" ")[1]]} {particleDict_jp["wa"]} {nounDict_jp[row["jp-r-2-passive"].split(" ")[4]]} {particleDict_jp["ni"]} {passiveDict_jp[verbDict_jp[row["jp-r-2-passive"].split(" ")[-2]]]}' , axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 103977.25it/s]


## JP - Unreal grammar 1:Add a negation at the end of the object in the real-jp-1 sentence

In [18]:
df['jp-u-1-negation'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[:3]) + " no " + " ".join(row['jp-r-1-sov'].split(" ")[3:]), axis=1)

df['jap-u-1-negation'] = df.progress_apply(lambda row: f'{" ".join(row["jap-r-1-sov"].split(" ")[:3])} {particleDict_jp["nai"]} {" ".join(row["jap-r-1-sov"].split(" ")[3:])}', axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 177875.12it/s]


## JP - Unreal grammar 2: Invert jp-real-1 sentence

In [19]:
df['jp-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[::-1]), axis=1)

df['jap-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['jap-r-1-sov'].split(" ")[::-1]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 245726.90it/s]


In [20]:
df

,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,en,en-r-1-subordinate,en-r-2-passive,...,it-r-2-passive,it-r-3-subordinate,it-u-1-negation,it-u-2-invert,it-u-3-gender,jp-r-1-sov,jp-r-2-passive,jp-r-3-subordinate,jp-u-1-negation,jp-u-2-invert
0,il cane porta il pesce,porta il pesce,Leela dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog carries the fish,Tom notices that the dog carries the fish,the fish is carried by the dog,...,kon fish is carried by kon dog,Tom notices that kon dog carries kon fish,kon dog carries kon no fish,fish kon carries dog kon,kon dog carries kon fish,the dog wa the fish o carries,the fish wa the dog ni to carry reru,Tom wa the dog ga the fish o carries to notices,the dog wa no the fish o carries,carries o fish the wa dog the
0,il cane tiene il pesce,tiene il pesce,Sheela afferma che il cane tiene il pesce,il pesce è tenuto dal cane,il cane tiene il no pesce,pesce il tiene cane il,il cane tiene il pesce,the dog holds the fish,Tom claims that the dog holds the fish,the fish is held by the dog,...,kon fish is held by kon dog,Tom claims that kon dog holds kon fish,kon dog holds kon no fish,fish kon holds dog kon,kon dog holds kon fish,the dog wa the fish o holds,the fish wa the dog ni to hold reru,Tom wa the dog ga the fish o holds to claims,the dog wa no the fish o holds,holds o fish the wa dog the
0,un cane prende il pesce,prende il pesce,Leela dice che un cane prende il pesce,il pesce è preso dal cane,un cane prende il no pesce,pesce il prende cane un,il cane prende il pesce,a dog takes the fish,Sheela sees that the dog takes the fish,the fish is taken by a dog,...,kon fish is taken by kon dog,Sheela sees that kon dog takes kon fish,kon dog takes kon no fish,fish kon takes dog kon,kon dog takes kon fish,a dog wa the fish o takes,the fish wa a dog ni to take reru,Sheela wa the dog ga the fish o takes to sees,a dog wa no the fish o takes,takes o fish the wa dog a
0,il cane porta il pesce,porta il pesce,Gomu dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog brings the fish,Tom claims that the dog brings the fish,the fish is brought by the dog,...,kon fish is brought by kon dog,Tom claims that kon dog brings kon fish,kon dog brings kon no fish,fish kon brings dog kon,kon dog brings kon fish,the dog wa the fish o brings,the fish wa the dog ni to bring reru,Tom wa the dog ga the fish o brings to claims,the dog wa no the fish o brings,brings o fish the wa dog the
0,un cane porta un topo,porta un topo,Harry dice che un cane porta un topo,un topo è portato da un cane,un cane porta un no topo,topo un porta cane un,un cane porta un topo,a dog carries a mouse,Harry sees that the dog carries the mouse,a mouse is carried by a dog,...,kon mouse is carried by kon dog,Harry sees that kon dog carries kon mouse,kon dog carries kon no mouse,mouse kon carries dog kon,kon dog carries kon mouse,a dog wa a mouse o carries,a mouse wa a dog ni to carry reru,Harry wa the dog ga the mouse o carries to sees,a dog wa no a mouse o carries,carries o mouse a wa dog a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,le ragazze leggono il capitolo,leggono il capitolo,Maria sa che le ragazze leggono il capitolo,il capitolo è letto dalle ragazze,le ragazze leggono il no capitolo,capitolo il leggono ragazze le,il ragazze leggono il capitolo,the girls read a chapter,Tom says that the girls read the chapter,a chapter is read by the girls,...,kon chapter is read by kars girls,Tom says that kars girls read kon chapter,kars girls read kon no chapter,chapter kon read girls kars,kars girls read kars chapter,the girls wa a chapter o read,a chapter wa the girls ni to read reru,Tom wa the girls ga the chapter o read to says,the girls wa no a chapter o read,read o chapter a

## JP - Unreal grammar add a after o + past tense

In [21]:
df['jp-u-3-past-tense'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[:-2] + [' o-ta '] + [infinitive[row['jp-r-1-sov'].split(" ")[-1]]]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 181573.54it/s]


In [22]:
df

ita ita-r-1-null_subject  \
0           il cane porta il pesce       porta il pesce   
0           il cane tiene il pesce       tiene il pesce   
0          un cane prende il pesce      prende il pesce   
0           il cane porta il pesce       porta il pesce   
0            un cane porta un topo        porta un topo   
..                             ...                  ...   
0   le ragazze leggono il capitolo  leggono il capitolo   
0       i ragazzi leggono il libro     leggono il libro   
0       i ragazzi leggono il poema     leggono il poema   
0      i ragazzi leggono la storia    leggono la storia   
0    i ragazzi leggono il capitolo  leggono il capitolo   

                             ita-r-2-subordinate  \
0          Leela dice che il cane porta il pesce   
0      Sheela afferma che il cane tiene il pesce   
0         Leela dice che un cane prende il pesce   
0           Gomu dice che il cane porta il pesce   
0           Harry dice che un cane porta un topo   
..                                           ...   
0    Maria sa che le ragazze leggono il capitolo   
0   Maria osserva che i ragazzi leggono il libro   
0         Gomu sa che i ragazzi leggono il poema   
0       Harry sa che i ragazzi leggono la storia   
0   Leela vede che i ragazzi leggono il capitolo   

                      ita-r-3-passive                   ita-u-1-negation  \
0         il pesce è portato dal cane          il cane porta il no pesce   
0          il pesce è tenuto dal cane          il cane tiene il no pesce   
0           il pesce è preso dal cane         un cane prende il no pesce   
0         il pesce è portato dal cane          il cane porta il no pesce   
0        un topo è portato da un cane           un cane porta un no topo   
..                                ...                                ...   
0   il capitolo è letto dalle ragazze  le ragazze leggono il no capitolo   
0        il libro è letto dai ragazzi      i ragazzi leggono il no libro   
0        il poema è letto dai ragazzi      i ragazzi leggono il no poema   
0       la storia è letto dai ragazzi     i ragazzi leggono la no storia   
0     il capitolo è letto dai ragazzi   i ragazzi leggono il no capitolo   

                    ita-u-2-invert                  ita-u-3-gender  \
0           pesce il porta cane il          il cane porta il pesce   
0           pesce il tiene cane il          il cane tiene il pesce   
0          pesce il prende cane un         il cane prende il pesce   
0           pesce il porta cane il          il cane porta il pesce   
0            topo un porta cane un           un cane porta un topo   
..                             ...                             ...   
0   capitolo il leggono ragazze le  il ragazze leggono il capitolo   
0       libro il leggono ragazzi i     il ragazzi leggono il libro   
0       poema il leggono ragazzi i     il ragazzi leggono il poema   
0      storia la leggono ragazzi i    la ragazzi leggono la storia   
0    capitolo il leggono ragazzi i  il ragazzi leggono il capitolo   

                           en                         en-r-1-subordinate  \
0    the dog carries the fish  Tom notices that the dog carries the fish   
0      the dog holds the fish     Tom claims that the dog holds the fish   
0        a dog takes the fish    Sheela sees that the dog takes the fish   
0     the dog brings the fish    Tom claims that the dog brings the fish   
0       a dog carries a mouse  Harry sees that the dog carries the mouse   
..                        ...                                        ...   
0    the girls read a chapter   Tom says that the girls read the chapter   
0      the boys read the book   John notices that the boys read the book   
0        the boys read a poem     Leela sees that the boys read the poem   
0       the boys read a story  Maria states that the boys read the story   
0   the boys read the chapter    Tom says that the boys read the chapter   

                     en-r-2-pa

## Non-grammatical sentences

In [23]:
def swap_words(sentence, col):
    sentence = sentence.split(" ")
    numWords = len(sentence)
    toSwap = [-1,-2]
    toSwapWords = set([sentence[toSwap[0]],sentence[toSwap[1]]])    
    swap1 = sentence[toSwap[0]]
    swap2 = sentence[toSwap[1]]
    sentence[toSwap[0]] = swap2
    sentence[toSwap[1]] = swap1
    return " ".join(sentence)

for col in list(df.columns):
    print(' Now processing.... ', col)
    df[f'ng-{col}'] = df.progress_apply(lambda row: swap_words(row[col], col), axis=1)

 Now processing....  ita


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 229964.54it/s]


 Now processing....  ita-r-1-null_subject


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 216308.34it/s]


 Now processing....  ita-r-2-subordinate


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 206673.64it/s]


 Now processing....  jp-u-2-invert


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 204994.27it/s]


 Now processing....  jp-u-3-past-tense


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 202159.13it/s]


In [24]:
gCols = [col for col in df.columns if not 'ng' in col]
ngCols = [col for col in df.columns if 'ng' in col]
df[ngCols]

,ng-ita,ng-ita-r-1-null_subject,ng-ita-r-2-subordinate,ng-ita-r-3-passive,ng-ita-u-1-negation,ng-ita-u-2-invert,ng-ita-u-3-gender,ng-en,ng-en-r-1-subordinate,ng-en-r-2-passive,...,ng-jp-r-1-sov,ng-jp-r-2-passive,ng-jp-r-3-subordinate,ng-jp-u-1-negation,ng-jp-u-2-invert,ng-jp-u-3-past-tense,ng-jap-r-1-sov,ng-jap-r-2-passive,ng-jap-u-1-negation,ng-jap-u-2-invert
0,il cane porta pesce il,porta pesce il,Leela dice che il cane porta pesce il,il pesce è portato cane dal,il cane porta il pesce no,pesce il porta il cane,il cane porta pesce il,the dog carries fish the,Tom notices that the dog carries fish the,the fish is carried by dog the,...,the dog wa the fish carries o,the fish wa the dog ni to reru carry,Tom wa the dog ga the fish o carries notices to,the dog wa no the fish carries o,carries o fish the wa the dog,the dog wa the fish o-ta carry to,犬 は 魚 運ぶ を,魚 は 犬 運ばれる に,犬 は 魚 ない 運ぶ を,運ぶ を 魚 犬 は
0,il cane tiene pesce il,tiene pesce il,Sheela afferma che il cane tiene pesce il,il pesce è tenuto cane dal,il cane tiene il pesce no,pesce il tiene il cane,il cane tiene pesce il,the dog holds fish the,Tom claims that the dog holds fish the,the fish is held by dog the,...,the dog wa the fish holds o,the fish wa the dog ni to reru hold,Tom wa the dog ga the fish o holds claims to,the dog wa no the fish holds o,holds o fish the wa the dog,the dog wa the fish o-ta hold to,犬 は 魚 持つ を,魚 は 犬 持たれる に,犬 は 魚 ない 持つ を,持つ を 魚 犬 は
0,un cane prende pesce il,prende pesce il,Leela dice che un cane prende pesce il,il pesce è preso cane dal,un cane prende il pesce no,pesce il prende un cane,il cane prende pesce il,a dog takes fish the,Sheela sees that the dog takes fish the,the fish is taken by dog a,...,a dog wa the fish takes o,the fish wa a dog ni to reru take,Sheela wa the dog ga the fish o takes sees to,a dog wa no the fish takes o,takes o fish the wa a dog,a dog wa the fish o-ta take to,犬 は 魚 とる を,魚 は 犬 とられる に,犬 は 魚 ない とる を,とる を 魚 犬 は
0,il cane porta pesce il,porta pesce il,Gomu dice che il cane porta pesce il,il pesce è portato cane dal,il cane porta il pesce no,pesce il porta il cane,il cane porta pesce il,the dog brings fish the,Tom claims that the dog brings fish the,the fish is brought by dog the,...,the dog wa the fish brings o,the fish wa the dog ni to reru bring,Tom wa the dog ga the fish o brings claims to,the dog wa no the fish brings o,brings o fish the wa the dog,the dog wa the fish o-ta bring to,犬 は 魚 もたらす を,魚 は 犬 持たれる に,犬 は 魚 ない もたらす を,もたらす を 魚 犬 は
0,un cane porta topo un,porta topo un,Harry dice che un cane porta topo un,un topo è portato da cane un,un cane porta un topo no,topo un porta un cane,un cane porta topo un,a dog carries mouse a,Harry sees that the dog carries mouse the,a mouse is carried by dog a,...,a dog wa a mouse carries o,a mouse wa a dog ni to reru carry,Harry wa the dog ga the mouse o carries sees to,a dog wa no a mouse carries o,carries o mouse a wa a dog,a dog wa a mouse o-ta carry to,犬 は マウス 運ぶ を,マウス は 犬 運ばれる に,犬 は マウス ない 運ぶ を,運ぶ を マウス 犬 は
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,le ragazze leggono capitolo il,leggono capitolo il,Maria sa che le ragazze leggono capitolo il,il capitolo è letto ragazze dalle,le ragazze leggono il capitolo no,capitolo il leggono le ragazze,il ragazze leggono capitolo il,the girls read chapter a,Tom says that the girls read chapter the,a chapter is read by girls the,...,the girls wa a chapter read o,a chapter wa the girls ni to reru read,Tom wa the girls ga the chapter o read says to,the girls wa no a chapter read o,read o chapter a wa the girls,the girls wa a chapter o-ta read to,女の子 は 章 読む を,章 は 女の子 読まれる に,女の子 は 章 ない 読む を,読む を 章 女の子 は
0,i ragazzi leggono libro il,leggono libro il,Maria osserva che i ragazzi leggono libro il,il libro è letto ragazzi dai,i ragazzi leggono il libro no,libro il leggono i ragazzi,il ragazzi leggono libro il,the boys read book the,John notices that the boys read book the,the book is read by boys the,...,the b

In [25]:
df.columns

Index(['ita', 'ita-r-1-null_subject', 'ita-r-2-subordinate', 'ita-r-3-passive',
       'ita-u-1-negation', 'ita-u-2-invert', 'ita-u-3-gender', 'en',
       'en-r-1-subordinate', 'en-r-2-passive', 'en-u-1-negation',
       'en-u-2-inversion', 'en-u-3-qsubordinate', 'it', 'it-r-1-null_subject',
       'it-r-2-passive', 'it-r-3-subordinate', 'it-u-1-negation',
       'it-u-2-invert', 'it-u-3-gender', 'jp-r-1-sov', 'jp-r-2-passive',
       'jp-r-3-subordinate', 'jp-u-1-negation', 'jp-u-2-invert',
       'jp-u-3-past-tense', 'jap-r-1-sov', 'jap-r-2-passive',
       'jap-u-1-negation', 'jap-u-2-invert', 'ng-ita',
       'ng-ita-r-1-null_subject', 'ng-ita-r-2-subordinate',
       'ng-ita-r-3-passive', 'ng-ita-u-1-negation', 'ng-ita-u-2-invert',
       'ng-ita-u-3-gender', 'ng-en', 'ng-en-r-1-subordinate',
       'ng-en-r-2-passive', 'ng-en-u-1-negation', 'ng-en-u-2-inversion',
       'ng-en-u-3-qsubordinate', 'ng-it', 'ng-it-r-1-null_subject',
       'ng-it-r-2-passive', 'ng-it-r-3-subordinat

In [26]:
len(df.columns)

52